<a href="https://colab.research.google.com/github/Matheus-brazao/Project-DIO-DatasetIA/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# Section 0 — Setup & Imports
# =========================
# Original: import keras (standalone, legacy), manual loaders
# Update: use TF 2.x (tf.keras), with TFDS or directory loader

import os
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

print("TF version:", tf.__version__)

# ================
# Section 1 — Config
# ================
# Change this to switch dataset quickly:
# - "tfds:cats_vs_dogs" (uses TFDS)
# - "dir:/content/drive/MyDrive/my_dataset" (directory with subfolders per class)
DATA_SOURCE = "tfds:cats_vs_dogs"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

# FAST_MODE: tiny run for quick sanity-check (few batches/epochs)
FAST_MODE = True
EPOCHS_BASELINE = 2 if FAST_MODE else 5
EPOCHS_TRANSFER = 2 if FAST_MODE else 5
STEPS_PER_EPOCH = 5 if FAST_MODE else 100
VAL_STEPS = 2 if FAST_MODE else 30
TEST_STEPS = 2 if FAST_MODE else 50

AUTOTUNE = tf.data.AUTOTUNE


# ==========================================
# Section 2 — Dataset loaders (TFDS or folder)
# ==========================================
def preprocess_baseline(image, label):
    # Baseline: simple [0,1] normalization
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def preprocess_for_vgg(image, label):
    # VGG16 expects its own preprocess_input
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)  # will be scaled by preprocess_input
    image = preprocess_input(image)     # center/scale as VGG16 expects
    return image, label

def load_tfds(name="cats_vs_dogs"):
    (ds_train, ds_val, ds_test), info = tfds.load(
        name,
        split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
        as_supervised=True,
        with_info=True
    )
    num_classes = info.features["label"].num_classes
    class_names = info.features["label"].names
    return ds_train, ds_val, ds_test, num_classes, class_names

def load_from_directory(dir_path):
    # Directory structure:
    # dir_path/
    #   class_a/ img1.jpg ...
    #   class_b/ ...
    train = tf.keras.preprocessing.image_dataset_from_directory(
        dir_path,
        validation_split=0.2, subset="training", seed=SEED,
        image_size=IMG_SIZE, batch_size=BATCH_SIZE
    )
    val = tf.keras.preprocessing.image_dataset_from_directory(
        dir_path,
        validation_split=0.2, subset="validation", seed=SEED,
        image_size=IMG_SIZE, batch_size=BATCH_SIZE
    )
    class_names = train.class_names
    num_classes = len(class_names)
    # Create a small test set from val (quick hack for demo)
    test = val.unbatch().batch(BATCH_SIZE)
    return train, val, test, num_classes, class_names

def build_pipelines(DATA_SOURCE):
    if DATA_SOURCE.startswith("tfds:"):
        name = DATA_SOURCE.split("tfds:")[1]
        ds_train, ds_val, ds_test, num_classes, class_names = load_tfds(name)
        # Two parallel pipelines: baseline and transfer (different preprocessing)
        train_base = ds_train.map(preprocess_baseline, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)
        val_base   = ds_val.map(preprocess_baseline,   num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)
        test_base  = ds_test.map(preprocess_baseline,  num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

        train_vgg = ds_train.map(preprocess_for_vgg, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)
        val_vgg   = ds_val.map(preprocess_for_vgg,   num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)
        test_vgg  = ds_test.map(preprocess_for_vgg,  num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

    elif DATA_SOURCE.startswith("dir:"):
        dir_path = DATA_SOURCE.split("dir:")[1]
        raw_train, raw_val, raw_test, num_classes, class_names = load_from_directory(dir_path)

        # Baseline pipelines
        train_base = raw_train.map(lambda x,y: (tf.cast(x, tf.float32)/255.0, y)).cache().prefetch(AUTOTUNE)
        val_base   = raw_val.map(lambda x,y: (tf.cast(x, tf.float32)/255.0, y)).cache().prefetch(AUTOTUNE)
        test_base  = raw_test.map(lambda x,y: (tf.cast(x, tf.float32)/255.0, y)).cache().prefetch(AUTOTUNE)

        # VGG pipelines
        train_vgg = raw_train.map(lambda x,y: (preprocess_input(tf.cast(x, tf.float32)), y)).cache().prefetch(AUTOTUNE)
        val_vgg   = raw_val.map(lambda x,y: (preprocess_input(tf.cast(x, tf.float32)), y)).cache().prefetch(AUTOTUNE)
        test_vgg  = raw_test.map(lambda x,y: (preprocess_input(tf.cast(x, tf.float32)), y)).cache().prefetch(AUTOTUNE)

    else:
        raise ValueError("DATA_SOURCE must start with 'tfds:' or 'dir:'")

    # FAST subsets (few steps) for quick runs
    train_base_small = train_base.take(STEPS_PER_EPOCH)
    val_base_small   = val_base.take(VAL_STEPS)
    test_base_small  = test_base.take(TEST_STEPS)

    train_vgg_small = train_vgg.take(STEPS_PER_EPOCH)
    val_vgg_small   = val_vgg.take(VAL_STEPS)
    test_vgg_small  = test_vgg.take(TEST_STEPS)

    return (train_base_small, val_base_small, test_base_small,
            train_vgg_small,  val_vgg_small,  test_vgg_small,
            num_classes, class_names)

(train_base, val_base, test_base,
 train_vgg,  val_vgg,  test_vgg,
 NUM_CLASSES, CLASS_NAMES) = build_pipelines(DATA_SOURCE)

print("Classes:", CLASS_NAMES, "| NUM_CLASSES:", NUM_CLASSES)


# =========================================
# Section 3 — Baseline model (from scratch)
# =========================================
def build_baseline(input_shape=IMG_SIZE+(3,), num_classes=2):
    model = Sequential([
        Conv2D(32, 3, activation="relu", input_shape=input_shape),
        MaxPooling2D(),
        Conv2D(64, 3, activation="relu"),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

baseline = build_baseline(num_classes=NUM_CLASSES)
baseline.summary()

history_base = baseline.fit(
    train_base,
    validation_data=val_base,
    epochs=EPOCHS_BASELINE,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VAL_STEPS,
    verbose=1
)


# =========================================
# Section 4 — Transfer learning (VGG16)
# =========================================
# Original: include_top=True then swap last Dense.
# Update: include_top=False and add our 2-class head.

base_model = VGG16(include_top=False, weights="imagenet", input_shape=IMG_SIZE+(3,))
for layer in base_model.layers:
    layer.trainable = False  # freeze feature extractor

inputs = tf.keras.Input(shape=IMG_SIZE+(3,))
x = inputs
x = base_model(x, training=False)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

transfer_model = Model(inputs, outputs, name="vgg16_transfer")
transfer_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])
transfer_model.summary()

history_transfer = transfer_model.fit(
    train_vgg,
    validation_data=val_vgg,
    epochs=EPOCHS_TRANSFER,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VAL_STEPS,
    verbose=1
)


# =========================================
# Section 5 — Quick comparison & test eval
# =========================================
plt.plot(history_base.history["val_accuracy"], label="baseline val_acc")
plt.plot(history_transfer.history["val_accuracy"], label="transfer val_acc")
plt.xlabel("epochs"); plt.ylabel("accuracy"); plt.legend(); plt.show()

print("Baseline test (subset):")
print(baseline.evaluate(test_base, verbose=0))

print("Transfer test (subset):")
print(transfer_model.evaluate(test_vgg, verbose=0))


# =========================================
# Section 6 — Tiny manual prediction (2 images)
# =========================================
# Take just 2 samples from the test_vgg pipeline and predict
for images, labels in test_vgg.take(1):
    preds = transfer_model.predict(images[:2])
    import numpy as np
    pred_ids = np.argmax(preds, axis=1)
    for i in range(2):
        plt.imshow((images[i].numpy() - images[i].numpy().min()) / (images[i].numpy().ptp()+1e-6))
        plt.title(f"Pred: {CLASS_NAMES[pred_ids[i]]} | True: {CLASS_NAMES[int(labels[i])]}")
        plt.axis("off")
        plt.show()
